# 1. Read in data.

In [2]:
# Import packages

import numpy as np
import pandas as pd
from scipy import stats
from datetime import datetime, timedelta

In [6]:
# Run generate_health_data.py and read in datasets

df_baseline = pd.read_csv("patient_baseline.csv", sep = '\t')
df_longitudinal = pd.read_csv("patient_longitudinal.csv", sep = '\t')
df_treatment = pd.read_csv("patient_treatment.csv", sep = '\t')

df_baseline.head()

,patient_id,age,sex,bmi,smoking,diabetes,bp_systolic,cholesterol,heart_rate
0,0,57.5,0,25.8,0,0,108.7,257.9,79.3
1,1,47.9,0,23.2,0,0,149.4,155.9,75.4
2,2,59.7,1,29.0,0,0,136.7,220.7,85.0
3,3,72.8,1,34.6,0,1,141.0,205.7,78.0
4,4,46.5,0,17.5,0,0,129.0,186.8,95.2
